In [1]:
!pip install datasets bitsandbytes trl huggingface-hub accelerate safetensors pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float32,
    bnb_4bit_use_double_quant= True
    )
repo = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(repo, quantization_config= bnb_config, device_map= "cuda:0")


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
print(model.get_memory_footprint()/1024/1024)

2104.1310424804688


In [5]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_

In [6]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r = 8, #. rank of LoRA - [4-16]
    bias = "none", # ["all", "lora_only"] - for train bias term
    lora_alpha = 16, # scalling factor
    lora_dropout = 0.05, # prevent overfit- used for regularisation
    target_modules = ["query_key_value", "o_proj", "qkv_proj", "gate_up_proj", "down_proj"],
    task_type = "CAUSAL_LM"

)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [7]:
print(model.get_memory_footprint()/1024/1024)

2528.2619018554688


In [8]:
print(model.get_base_model)

<bound method PeftModel.get_base_model of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Mo

In [9]:
trainable_params, total_params = model.get_nb_trainable_parameters()
percentage = (trainable_params / total_params) * 100

print(f"Trainable Parameters: {trainable_params:,}")
print(f"Total Parameters: {total_params:,}")
print(f"Percentage Trainable: {percentage:.2f}%")

Trainable Parameters: 12,582,912
Total Parameters: 3,833,662,464
Percentage Trainable: 0.33%


In [40]:
dataset1 = load_dataset("sweatSmile/FinNLP-QA-1.0", split="train")
dataset1

Dataset({
    features: ['COMPANY_ID', 'QUERY', 'ANSWER', 'CONTEXT', '__index_level_0__'],
    num_rows: 37831
})

In [19]:
dataset = dataset.remove_columns(["__index_level_0__"])

In [20]:
dataset[0]

{'COMPANY_ID': 'STERTOOLS_2023.txt_19',
 'QUERY': "What is the company's approach to supply chain management?",
 'ANSWER': "The company's approach to supply chain management focuses on enabling intelligent manufacturing and predictive maintenance to drive efficiencies on the shop floor.",
 'CONTEXT': 'NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Confer

In [21]:
dataset = dataset.select(range(1000))

In [22]:
# 3. Create prompt and completion
def build_prompt(example):
    symbol = example["COMPANY_ID"].split("_")[0]
    year = example["COMPANY_ID"].split("_")[1]
    context = example["CONTEXT"].strip()
    question = example["QUERY"].strip()
    return f"Context: Symbol: {symbol} | Year: {year} | {context}\n\nQuery: {question}"

dataset = dataset.map(lambda x: {
    "prompt": build_prompt(x),
    "completion": x["ANSWER"].strip()
})
dataset = dataset.remove_columns(["COMPANY_ID", "QUERY", "ANSWER", "CONTEXT"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
dataset[0]

{'prompt': "Context: Symbol: STERTOOLS | Year: 2023.txt | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclu

In [24]:
# 4. Convert to chat format
chat_dataset = dataset.map(lambda x: {
    "messages": [
        {"role": "user", "content": x["prompt"]},
        {"role": "assistant", "content": x["completion"]}
    ]
})
chat_dataset = chat_dataset.remove_columns(["prompt", "completion"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
tokenizer = AutoTokenizer.from_pretrained(repo)
tokenizer.chat_template

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

In [27]:
# Apply chat template to an example from chat_dataset
example = chat_dataset[0]
print(tokenizer.apply_chat_template(example["messages"], tokenize=False))

<|user|>
Context: Symbol: STERTOOLS | Year: 2023.txt | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclusiv

In [28]:
sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': True},  # <-- fixed here
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True,
    max_seq_length=64,
    packing=True,
    num_train_epochs=10,
    learning_rate=3e-4,
    optim='paged_adamw_8bit',
    logging_steps=10,
    logging_dir="/content/drive/MyDrive/phi3-mini-finance-nlp/logs",
    output_dir="/content/drive/MyDrive/phi3-mini-finance-nlp/adapter",
    report_to="none"
)

In [29]:
trainer=SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=sft_config
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:412: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:458: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2'. Packing flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to cross-contamination between batches. To avoid this, ei

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [30]:
dl=trainer.get_train_dataloader()
batch = next(iter(dl))

In [31]:
batch['input_ids'][0], batch['labels'][0]

(tensor([15228, 29901, 23858,  ..., 11408, 29879,  7225], device='cuda:0'),
 tensor([15228, 29901, 23858,  ..., 11408, 29879,  7225], device='cuda:0'))

In [32]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.428500
20,1.084400
30,1.117800
40,1.137200
50,1.058500
60,0.878000
70,0.875200
80,0.766800
90,0.664200
100,0.691000


TrainOutput(global_step=630, training_loss=0.3930223200056288, metrics={'train_runtime': 1056.4646, 'train_samples_per_second': 9.466, 'train_steps_per_second': 0.596, 'total_flos': 1.434302152704e+16, 'train_loss': 0.3930223200056288})

In [ ]:
# def gen_prompt(tokenizer, context, query, symbol="XYZ", year="2023"):
#     prompt_text = f"Context: Symbol: {symbol} | Year: {year} | {context.strip()}\n\nQuery: {query.strip()}"
#     chat_format = [{"role": "user", "content": prompt_text}]
#     prompt = tokenizer.apply_chat_template(chat_format, tokenize=False, add_generation_prompt=True)
#     return prompt


In [43]:
def gen_prompt(tokenizer, query, ticker, year, df):
    # Look up the row in your context dataset
    match = df[df["COMPANY_ID"].str.contains(f"{ticker}_{year}")]

    if match.empty:
        raise ValueError(f"No context found for {ticker}_{year}")

    context = match.iloc[0]["CONTEXT"]

    # Construct prompt with context + query
    full_prompt = f"Context: Symbol: {ticker} | Year: {year} | {context.strip()}\n\nQuery: {query.strip()}"
    messages = [{"role": "user", "content": full_prompt}]

    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [46]:
def generate(model, tokenizer, prompt, max_new_tokens=128, skip_special_tokens=True):
    model.eval()
    tokenized_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    generation_output = model.generate(
        **tokenized_input,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.batch_decode(generation_output, skip_special_tokens=skip_special_tokens)
    return decoded[0]


In [44]:
context_df = dataset1.to_pandas()

In [53]:
query = "What is the company's approach to supply chain management?"
ticker = "STERTOOLS"
year = "2023"

prompt = gen_prompt(tokenizer, query, ticker, year, context_df)
output = generate(model, tokenizer, prompt)


import re

def extract_answer(output):
    match = re.search(r"<\|assistant\|>\n(.*?)(<\|end\|>|$)", output, re.DOTALL)
    return match.group(1).strip() if match else output.strip()

print(extract_answer(output))




# Optional: Extract just the answer
# print(output.split("<|assistant|>\n")[-1].strip().replace("<|end|>", ""))

Context: Symbol: STERTOOLS | Year: 2023 | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclusive Remote e vo

In [54]:
def clean_answer(output: str) -> str:
    if "<|assistant|>" in output:
        output = output.split("<|assistant|>")[-1]
    if "<|end|>" in output:
        output = output.split("<|end|>")[0]
    return output.strip()

# Call it like this:
print(clean_answer(output))

Context: Symbol: STERTOOLS | Year: 2023 | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclusive Remote e vo

In [ ]:
# # Example context and query
# context = "The company reported a strong financial year with net profit rising 20% to ₹325 crore from ₹270 crore last year."
# query = "What is the net profit of the company?"
# symbol = "ABC"
# year = "2023"

# prompt = gen_prompt(tokenizer, context, query, symbol, year)
# print(prompt)


In [55]:
def generate(model, tokenizer, prompt, max_new_tokens=128, skip_special_tokens=True):
    model.eval()
    tokenized_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    generation_output = model.generate(
        **tokenized_input,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.batch_decode(generation_output, skip_special_tokens=skip_special_tokens)
    return decoded[0]


In [58]:
def extract_clean_answer(output: str) -> str:
    # Split after the last question mark to remove the prompt
    if "?" in output:
        answer_part = output.split("?")[-1]
    else:
        answer_part = output
    return answer_part.strip().replace("<|end|>", "")


In [59]:
output = generate(model, tokenizer, prompt)
answer = extract_clean_answer(output)
print("Answer:", answer)

Answer: Sterling Tools Limited follows a comprehensive approach towards supply chain management that includes effective procurement processes, efficient inventory management, reliable logistics, and strong relationships with suppliers. The company continuously monitors and evaluates its supply chain performance to identify areas for improvement and implement necessary measures. Sterling Tools Limited believes that a well-managed supply chain is critical to achieving operational excellence and ensuring customer satisfaction.


In [56]:
output = generate(model, tokenizer, prompt)
print(output)

Context: Symbol: STERTOOLS | Year: 2023 | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclusive Remote e vo

In [57]:
answer = output.split("<|assistant|>\n")[-1].strip().replace("<|end|>", "")
print("Answer:", answer)

Answer: Context: Symbol: STERTOOLS | Year: 2023 | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclusive Rem

In [ ]:
prompt = gen_prompt(...)
output = generate(model, tokenizer, prompt)


In [60]:
def gen_prompt(tokenizer, query, ticker, year, df):
    # Look up the row in your context dataset
    match = df[df["COMPANY_ID"].str.contains(f"{ticker}_{year}")]

    if match.empty:
        raise ValueError(f"No context found for {ticker}_{year}")

    context = match.iloc[0]["CONTEXT"]

    # Construct prompt with context + query
    full_prompt = f"Context: Symbol: {ticker} | Year: {year} | {context.strip()}\n\nQuery: {query.strip()}"
    messages = [{"role": "user", "content": full_prompt}]

    # ✅ Return the prompt!
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [61]:
def generate(model, tokenizer, prompt, max_new_tokens=128, skip_special_tokens=True):
    model.eval()
    tokenized_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
    generation_output = model.generate(
        **tokenized_input,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.batch_decode(generation_output, skip_special_tokens=skip_special_tokens)
    return decoded[0]


In [62]:
prompt = gen_prompt(tokenizer, query, ticker, year, context_df)
output = generate(model, tokenizer, prompt)
print(output)

Context: Symbol: STERTOOLS | Year: 2023 | NEAPS National Stock Exchange India Limited Exchange Plaza Bandra Kurla Complex Sandra E Security Code STERTOOLS Date 2611 August 2023 ub lotico tho 2022 2023 Ro 2015 D ar Sir STERLING TOOLS LIMITED CIN L29222DL 1979PLC009668 WORKS 5 DLF Industrial state Faridabad 121 003 Haryana India Tel 91 227 0621 5551 53 Fax 91 129 227 7359 com website stlfastcners corn Listing Centre Secretary BSE Limited 25111 Floor Towers Dalal Street Mumbai 400001 Security Code 530759 reference captioned subject submitting herewith Notic Annual General Meeting AGM Annual Report inancial Year 2022 2023 sent Shareholders lectronic Mode Annual General Meeting Company held Monday 181h September 2023 1 ST Video Conferencing Audio Visual means tout Events Dav Date Time 1ST Relevant Date Record Date date Monday 11 th September 2023 NA vote AGM Resolution Book Closure Date AGM Final Tuesday 12111 September 2023 Monday NA Dividend 18111 September 2023 days inclusive Remote e vo

In [63]:
trainer.save_model('phi3-mini-finance-adapter')

In [1]:
!pip install huggingface_hub
huggingface-cli login()


SyntaxError: invalid syntax (ipython-input-1-2057043595.py, line 2)